In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mpmath as mp
from pathlib import Path

### Import `Cymetric` sampling from the Weierstrass cubic in $\mathbb{P}^2$

In [ ]:
fname = "/home/habjan.e/CY_metric/ips_sampling/cymetric/weierstrass_cubic/weierstrass_sampling.npy"

Z = np.load(fname)

### Weierstrass coeficients from the data above 

a = 1.0
b = -4.0
c = 3.151212 * 60

### Make a plot showing the real components of $Z_2 / Z_0$ and $Z_1 / Z_0$

In [ ]:
# Keep the Z0!=0 patch and go to affine coordinates
mask = np.abs(Z[:,0]) > 1e-14
Z = Z[mask]
x = Z[:,1] / Z[:,0]
y = Z[:,2] / Z[:,0]

# Real 2D projection (paper-style square)
X = np.real(x)
Y = np.real(y)

# (Optional) clip extreme outliers for a tighter square view
q = 0.995
lo_x, hi_x = np.quantile(X, 1-q), np.quantile(X, q)
lo_y, hi_y = np.quantile(Y, 1-q), np.quantile(Y, q)

#plt.figure(figsize=(4,14))
plt.scatter(np.clip(X, lo_x, hi_x), np.clip(Y, lo_y, hi_y), c='k', s=1.0, alpha=0.8)
#plt.gca().set_aspect('equal', 'box')
plt.xlabel(r'Re$(Z_1/Z_0)$', fontsize = 15); plt.ylabel(r'Re$(Z_2/Z_0)$', fontsize = 15)
plt.xlim(lo_x, hi_x); plt.ylim(lo_y, hi_y)
#plt.title('Weierstrass cubic: affine (real) projection')
plt.tight_layout()
plt.show()

### We will homogenize the data by setting $x = Z_1 / Z_0$ and $y = Z_2 / Z_0$. This is what is called an affine transformation of our coordinates. From the the Weierstrass cubic we have that: 

$$
0 = a Z_2^2 Z_0 + b Z_1^3 + c Z_1 Z_0^2
$$

$$
\rightarrow 0 = a (Z_0 y)^2 Z_0 + b (Z_0 x)^3 + c (Z_0 x) Z_0^2
$$

$$
\rightarrow 0 = a y^2 + b x^3 + c x
$$

$$
\rightarrow y^2 = - (b / a) x^3 - (c / a) x
$$

### Dehomogenize the data that we sampled

In [ ]:
mask = np.abs(Z[:,0]) > 1e-14
x = (Z[mask,1] / Z[mask,0]).astype(np.complex128)
y = (Z[mask,2] / Z[mask,0]).astype(np.complex128)

### Constants for Jacobi-uniformization

In [ ]:
# --- constants for g3=0 (lemniscatic) ---
c_real = float(np.real(c))       # c > 0 in your setup
rt = mp.sqrt(c_real)             # √c (real, positive)
a = rt/2                         # e1=+a, e2=0, e3=-a
alpha = c_real**0.25             # α = (e1-e3)^{1/2} = c^{1/4}
m = mp.mpf('0.5')                # k^2 = 1/2
K  = mp.ellipk(m)
Kp = mp.ellipk(1-m)

### We set the homogenized polynomial such that: 

$$
\rightarrow y^2 = - (b / a) x^3 - (c / a) x = - (b / a) (x - e_1) (x - e_2) (x - e_3)
$$

With:

$$
e_1 = \frac{1}{2} \sqrt{c}, e_2 = 0, e_3 = - \frac{1}{2} \sqrt{c}
$$

And we will have the half periods defined as:

$$
2 \omega_1 = \cfrac{2 K}{c^{1/4}} \in \mathbb{R}\text{   and    }2 \omega_2 = i \cfrac{2 K^\prime}{c^{1/4}} = i \cfrac{2 K}{c^{1/4}}
$$

where $\omega_2 / \omega_1 = i$, which gives us a square torus. 

### Use Jacobi Elliptic functions to trasnform (x, y) $\rightarrow$ U (Jacobi elliptic parameter)

In [ ]:
# Jacobi helpers for the older mpmath API: ellipfun(kind, u, m)
def jacobi_sn(u, m): return mp.ellipfun('sn', u, m)
def jacobi_cn(u, m): return mp.ellipfun('cn', u, m)
def jacobi_dn(u, m): return mp.ellipfun('dn', u, m)

def inv_u(xi, yi, *, m=m, alpha=alpha, rt=rt, a=a):
    """Invert x -> u on y^2 = 4x^3 - c x (g3=0) using Jacobi sn."""
    # sn^2(αu) = √c / (x + √c/2)
    s2   = rt / (xi + a)          # can be complex
    s    = mp.sqrt(s2)            # principal branch
    Uhat = mp.ellipf(mp.asin(s), m)  # Uhat = α u  (inverse sn)

    # Predict y from Uhat to choose the correct sheet/sign
    sn = jacobi_sn(Uhat, m)
    cn = jacobi_cn(Uhat, m)
    dn = jacobi_dn(Uhat, m)
    y_pred = -(alpha**3) * (cn * dn) / (sn**3)

    u = Uhat / alpha
    if abs(y_pred - yi) > abs(-y_pred - yi):
        u = -u

    return complex(u)

U = np.array([inv_u(xi, yi) for xi, yi in zip(x, y)], dtype=np.complex128)

### Obtain the Weierstrass/Abel-Jacobi uniformization coordinate

In [ ]:
w1, w2 = K/alpha, 1j*Kp/alpha   # half-periods

M = np.array([[ (2*w1).real, (2*w2).real ],
              [ (2*w1).imag, (2*w2).imag ]], dtype=float)

ab = (np.linalg.inv(M) @ np.vstack([U.real, U.imag])).T

### Plot the distribution of random points on the Weierstrass cubic from `cymetric` in uniformization coordinates

In [ ]:
plt.figure(figsize=(6, 6))

plt.scatter(ab[:,0], ab[:,1], c='k', s=2, alpha=0.7)

plt.gca().set_aspect('equal', 'box')
plt.tight_layout(); plt.show()